In [10]:
# ✅ Task 8: Spatial & Geospatial Data Visualization using Banking Loan Dataset

import pandas as pd
import folium
from folium.plugins import HeatMap
import random

# Load dataset
df = pd.read_csv("/content/banking_loan_applications.csv")

# Show columns
print("Columns:", df.columns.tolist())

# ✅ Step 1: Add sample 'City' column if missing
if 'City' not in df.columns:
    tamilnadu_cities = ["Chennai", "Madurai", "Coimbatore", "Trichy", "Salem", "Erode", "Vellore", "Tirunelveli"]
    df['City'] = [random.choice(tamilnadu_cities) for _ in range(len(df))]

# ✅ Step 2: Add city coordinates (Latitude & Longitude)
city_coords = {
    "Chennai": [13.0827, 80.2707],
    "Madurai": [9.9252, 78.1198],
    "Trichy": [10.7905, 78.7047],
    "Salem": [11.6643, 78.1460],
    "Tirunelveli": [8.7139, 77.7567],
    "Erode": [11.3410, 77.7172],
    "Vellore": [12.9165, 79.1325]
}

df['Coordinates'] = df['City'].map(city_coords)
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

# Drop missing coordinates if any
df = df.dropna(subset=['Latitude', 'Longitude'])

# -------------------------------------------------------------------------
# 🗺️ PART 1 — SPATIAL DATA VISUALIZATION
# -------------------------------------------------------------------------
# Spatial = Data that is *mapped to locations* (cities, points, regions) without using geocoordinates

spatial_map = folium.Map(location=[11.0, 78.0], zoom_start=7, tiles="cartodb positron")

for city, group in df.groupby('City'):
    loan_count = len(group)
    approval_rate = (group['Loan_Status'].str.lower() == 'approved').mean() * 100
    popup = f"<b>City:</b> {city}<br><b>Total Loans:</b> {loan_count}<br><b>Approval Rate:</b> {approval_rate:.1f}%"
    folium.Marker(
        location=city_coords[city],
        popup=popup,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(spatial_map)

spatial_map.save("/content/spatial_loan_map.html")
print("✅ Spatial Map saved as spatial_loan_map.html")
spatial_map



Columns: ['Application_ID', 'Application_Date', 'Gender', 'Age', 'Employment_Type', 'Annual_Income', 'Loan_Amount', 'Loan_Purpose', 'Credit_Score', 'Loan_Status']
✅ Spatial Map saved as spatial_loan_map.html


In [ ]:
# -------------------------------------------------------------------------
# 🌍 PART 2 — GEOSPATIAL DATA VISUALIZATION
# -------------------------------------------------------------------------
# Geospatial = Data that includes actual geographic coordinates (Lat, Long) + attributes

geo_map = folium.Map(location=[11.0, 78.0], zoom_start=7, tiles="cartodb positron")

# Plot loan data with color-coded status
for _, row in df.iterrows():
    color = 'green' if str(row.get('Loan_Status', '')).lower() == 'approved' else 'red'
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"City: {row['City']}<br>Loan: ₹{row.get('Loan_Amount', 'N/A')}<br>Status: {row.get('Loan_Status', 'Unknown')}"
    ).add_to(geo_map)

# Add HeatMap based on loan amounts
if 'Loan_Amount' in df.columns:
    heat_data = [[row['Latitude'], row['Longitude'], float(row['Loan_Amount'])] for _, row in df.iterrows()]
    HeatMap(heat_data, radius=20, blur=15).add_to(geo_map)

geo_map.save("/content/geospatial_loan_map.html")
print("✅ Geospatial Map saved as geospatial_loan_map.html")

geo_map


✅ Geospatial Map saved as geospatial_loan_map.html
